In [1]:
from json import dump, load
from os import getenv, listdir
import pandas as pd
from requests import get
from time import sleep

In [2]:
AV_API = getenv('AV_API')

In [3]:
currencies = pd.read_csv('./currency_list.csv')
currencies

,currency_code,currency_name
0,1ST,FirstBlood
1,2GIVE,GiveCoin
2,808,808Coin
3,AAVE,Aave
4,ABT,ArcBlock
...,...,...
570,ZEN,ZenCash
571,ZET,Zetacoin
572,ZIL,Zilliqa
573,ZLA,Zilla


In [4]:
try:
    allData = pd.read_csv('./data/all_data.csv')
except FileNotFoundError:
    allData = pd.DataFrame()
else:
    allData = allData.iloc[: , 1:]

for timeFrame in ('daily', 'weekly', 'monthly'):
    for symbol in currencies['currency_code']:

        if f'{timeFrame}_{symbol}.csv' in listdir('./data/'):
            continue

        url = f'https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_{timeFrame.upper()}&symbol={symbol}&market=USD&apikey={AV_API}&datatype=csv'
        
        try:
            response = get(url, timeout = 60)
        except:
            print(f'{timeFrame} {symbol} connection error.')
            continue
        
        sleep(15)

        if 'API' in response.text:
            print(f'{timeFrame} {symbol} reached to API limit.')
            continue

        print(f'{timeFrame} {symbol} {response}')

        if response.status_code == 200:
            with open(f'./data/{timeFrame}_{symbol}.csv', 'wb') as dataFile:
                dataFile.write(response.content)
        else:
            print(f'{timeFrame} {symbol} not responding correctly.')
            continue

        data = pd.read_csv(f'./data/{timeFrame}_{symbol}.csv')
        data['time_frame'] = timeFrame
        data['symbol'] = symbol

        allData = pd.concat([allData, data])
        allData.reset_index(drop = True, inplace = True)
        allData.to_csv('./data/all_data.csv')

allData

daily 1ST reached to API limit.
daily 2GIVE reached to API limit.
daily 808 reached to API limit.
daily AAVE <Response [200]>
daily ABT reached to API limit.
daily ABY reached to API limit.
daily AC reached to API limit.
daily ACT reached to API limit.
daily ADA <Response [200]>
daily ADT reached to API limit.
daily ADX <Response [200]>
daily AE reached to API limit.
daily AEON reached to API limit.
daily AGI reached to API limit.
daily AGRS reached to API limit.
daily AI reached to API limit.
daily AID reached to API limit.
daily AION <Response [200]>
daily AIR reached to API limit.
daily AKY reached to API limit.
daily ALGO <Response [200]>
daily ALIS reached to API limit.
daily AMBER reached to API limit.
daily AMP <Response [200]>
daily AMPL reached to API limit.
daily ANC <Response [200]>
daily ANT <Response [200]>
daily APPC reached to API limit.
daily APX reached to API limit.
daily ARDR <Response [200]>
daily ARK reached to API limit.
daily ARN reached to API limit.
daily AST r

,timestamp,open (USD),high (USD),low (USD),close (USD),open (USD).1,high (USD).1,low (USD).1,close (USD).1,volume,market cap (USD),time_frame,symbol,{}
0,2023-05-08,68.7000,69.2000,68.2000,69.0000,68.7000,69.2000,68.2000,69.0000,2.171491e+03,2.171491e+03,daily,AAVE,NaN
1,2023-05-07,69.7000,70.2000,68.2000,68.7000,69.7000,70.2000,68.2000,68.7000,2.555457e+04,2.555457e+04,daily,AAVE,NaN
2,2023-05-06,73.4000,74.1000,68.5000,69.8000,73.4000,74.1000,68.5000,69.8000,6.431654e+04,6.431654e+04,daily,AAVE,NaN
3,2023-05-05,70.8000,74.3000,70.0000,73.5000,70.8000,74.3000,70.0000,73.5000,5.979686e+04,5.979686e+04,daily,AAVE,NaN
4,2023-05-04,71.8000,72.1000,70.2000,70.8000,71.8000,72.1000,70.2000,70.8000,2.978446e+04,2.978446e+04,daily,AAVE,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118686,2021-01-31,0.3550,0.8000,0.3509,0.7064,0.3550,0.8000,0.3509,0.7064,7.008664e+08,7.008664e+08,monthly,ZRX,NaN
118687,2020-12-31,0.4294,0.4483,0.3100,0.3548,0.4294,0.4483,0.3100,0.3548,1.941776e+08,1.941776e+08,monthly,ZRX,NaN
118688,2020-11-30,0.3214,0.5285,0.3057,0.4292,0.3214,0.5285,0.3057,0.4292,2.325502e+08,2.325502e+08,monthly,ZRX,NaN
118689,2020-10-31,0.4098,0.4349,0.3150,0.3219,0.4098,0.4349,0.3150,0.3219,1.249353e+08,1.249353e+08,monthly,ZRX,NaN


In [45]:
allData = pd.read_csv('./data/all_data.csv')
allData.drop(columns = ['Unnamed: 0', 'open (USD).1', 'high (USD).1', 'low (USD).1', 'close (USD).1', '{}'], inplace = True)
allData.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'marketcap', 'timeframe', 'symbol']
allData = allData[['timeframe', 'symbol', 'timestamp', 'open', 'high', 'low', 'close', 'volume', 'marketcap']]
allData

,timeframe,symbol,timestamp,open,high,low,close,volume,marketcap
0,daily,AAVE,2023-05-08,68.7000,69.2000,68.2000,69.0000,2.171491e+03,2.171491e+03
1,daily,AAVE,2023-05-07,69.7000,70.2000,68.2000,68.7000,2.555457e+04,2.555457e+04
2,daily,AAVE,2023-05-06,73.4000,74.1000,68.5000,69.8000,6.431654e+04,6.431654e+04
3,daily,AAVE,2023-05-05,70.8000,74.3000,70.0000,73.5000,5.979686e+04,5.979686e+04
4,daily,AAVE,2023-05-04,71.8000,72.1000,70.2000,70.8000,2.978446e+04,2.978446e+04
...,...,...,...,...,...,...,...,...,...
118686,monthly,ZRX,2021-01-31,0.3550,0.8000,0.3509,0.7064,7.008664e+08,7.008664e+08
118687,monthly,ZRX,2020-12-31,0.4294,0.4483,0.3100,0.3548,1.941776e+08,1.941776e+08
118688,monthly,ZRX,2020-11-30,0.3214,0.5285,0.3057,0.4292,2.325502e+08,2.325502e+08
118689,monthly,ZRX,2020-10-31,0.4098,0.4349,0.3150,0.3219,1.249353e+08,1.249353e+08


In [58]:
allData['symbol'].unique()

array(['AAVE', 'ADA', 'ADX', 'AION', 'ALGO', 'AMP', 'ANC', 'ANT', 'ARDR',
       'ATM', 'ATOM', 'AVAX', 'BAND', 'BAT', 'BCC', 'BCH', 'BLZ', 'BNB',
       'BNT', 'BTC', 'BTG', 'BTS', 'BTT', 'BUSD', 'CAKE', 'COMP', 'CTXC',
       'CVC', 'DAI', 'DAR', 'DASH', 'DATA', 'DCR', 'DENT', 'DGB', 'DNT',
       'DOGE', 'DOT', 'EGLD', 'ELF', 'ENJ', 'EOS', 'ETC', 'ETH', 'FIL',
       'FTT', 'FUN', 'GAS', 'GNO', 'GRT', 'GTC', 'GTO', 'GXS', 'ICX',
       'IOST', 'IOTA', 'IOTX', 'KMD', 'KNC', 'KSM', 'LEND', 'LINK',
       'LOOM', 'LRC', 'LSK', 'LTC', 'LUNA', 'MANA', 'MATIC', 'MCO',
       'MITH', 'MKR', 'MLN', 'NANO', 'NBT', 'NEBL', 'NEO', 'NMR', 'NPXS',
       'NULS', 'OAX', 'OMG', 'ONT', 'ORN', 'POLY', 'POWR', 'QTUM',
       'QUICK', 'REP', 'REQ', 'RLC', 'RUNE', 'SC', 'SHIB', 'SOL', 'SNX',
       'STEEM', 'STORJ', 'STORM', 'STRAT', 'STX', 'SYS', 'THETA', 'TRIBE',
       'TRX', 'TUSD', 'UNI', 'UST', 'UTK', 'VEN', 'VIB', 'WAN', 'WAVES',
       'WBTC', 'XEM', 'XLM', 'XRP', 'XTZ', 'XVG', 'XZC', 'ZEC', 'Z